In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ROOT
import os
ROOT.enableJSVis()
c1 = ROOT.TCanvas()
ROOT.gSystem.Load("/usr/local/software/quadis/build/lib/libquadis.so")   

Welcome to JupyROOT 6.10/00


0

In [2]:
infile = ROOT.TFile("/home/vasu/PPC/PPC_SigGen/SCAN100004/SCAN100004_RUN13.root")
tree = infile.Get("T1")
nevent = ROOT.NEvent()
tree.SetBranchAddress("NEvent",nevent)
tree.GetEntry(4043)
npulse = nevent.GetPulse(0)
sP = npulse.GetSamplingPeriod()

#Initialize the fitter
fitter = ROOT.NAPulseFitter("/home/vasu/PPC/PPC_SigGen/SignalLib_root/new_OutputRZperfect.root","T1","NEvent",1)

#Parameters to pre-process pulse
baselines = 1000
taus = 7100
rises=750
gaps=225
pulse = fitter.GetPreProcessedPulse(npulse, False, baselines, taus, rises, gaps)# true applies pole-zero

blrPulse = ROOT.NPulse()
blrFilter = ROOT.NFBaselineRemover(1000*sP) #use 1000 samples to calculate the baseline
blrFilter.Filter(pulse,blrPulse)# apply the filter
baseline = blrFilter.GetCalcParameterByName("Baseline")
# print("the baseline of the pulse was",baseline)

trapPulse = ROOT.NPulse() #to hold the filtered result
trapFilter = ROOT.NFTrapezoid(20*sP,10*sP,6750,baseline) #tau of about 6000 samples
trapFilter.SetBaseline(baseline)
trapFilter.Filter(pulse,trapPulse)
# print("The maximum of the trapezoid is",trapFilter.GetCalcParameterByName("Max"))

blrPC = ROOT.NPCBaseline(2000*sP) 
alist = blrPC.GetParameterNames() #get all of the available parameters
blrPC.CalculateParameters(trapPulse) #calculate all of the parameters
baseline1 = blrPC.GetCalcParameterByName("Baseline")
print(baseline1)

pulseh = pulse.GetTH1Dmus()
traph = trapPulse.GetTH1Dmus()
traph.SetLineColor(3)
pulseh.Draw()
traph.Draw("Same")
c1.Draw()

0.000320841452199


In [3]:
sgdPulse = ROOT.NPulse()
sgFilter = ROOT.NFSavitzkyGolay()
sgFilter.SetSGType(5);
sgFilter.Filter(pulse,sgdPulse)

blrPC2 = ROOT.NPCBaseline(2000*sP) 
alist2 = blrPC2.GetParameterNames() #get all of the available parameters
blrPC2.CalculateParameters(sgdPulse) #calculate all of the parameters
baseline2 = blrPC2.GetCalcParameterByName("Baseline")
print(baseline2)

pulseh = pulse.GetTH1Dmus()
sgdh = sgdPulse.GetTH1Dmus()
sgdh.SetLineColor(1)
pulseh.Draw()
sgdh.Draw("same")

c1.Draw()

1.2177586775e-05
